In [ ]:
import sys
!{sys.executable} -m pip install sagemaker --quiet --upgrade

In [ ]:
import boto3
import os
import sagemaker
import tensorflow as tf

print(sagemaker.__version__)
sess = sagemaker.session.Session()
bucket = sess.default_bucket() 
region = boto3.Session().region_name

In [ ]:
!wget -q https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/local_mode_setup.sh
!wget -q https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/daemon.json    
!/bin/bash ./local_mode_setup.sh

In [ ]:
from sagemaker.tensorflow import TensorFlow

git_config = {'repo': 'https://github.com/Cellular-Longevity/data-processing/', 
              'branch': 'main'}

train_instance_type = 'local'
hyperparameters = {'epochs': 2, 'batch-size': 128, 'learning-rate': 0.01, 'imgsize':224, 'lds' = False}

local_estimator = TensorFlow(git_config=git_config,
                             source_dir='src/notebooks/-----',
                             entry_point='train.py',
                             instance_type=train_instance_type,
                             instance_count=1,
                             hyperparameters=hyperparameters,
                             role=sagemaker.get_execution_role(),
                             base_job_name='tf-2-workflow',
                             framework_version='2.3.1',
                             py_version='py37',
                             script_mode=True)

In [ ]:
inputs = {'images' : 's3://',
          'traindf': 's3://',
          'valdf'  : 's3://'}

local_estimator.fit(inputs)